In [3]:
# 数据获取
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
import numpy as np

# 获取乳腺癌数据集
cancers = load_breast_cancer()
# 切割80%训练和20%的测试数据
cancers_X_train, cancers_X_test, cancers_Y_train, cancers_Y_test = train_test_split(
                                        cancers.data, cancers.target, test_size=0.2)

> 随机森林查找最重要的特征

In [19]:
from sklearn.ensemble import RandomForestClassifier
import numpy as np

# 创建随机森林分类器对象
rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)

rf_classifier.fit(cancers_X_train, cancers_Y_train)

feature_importances = rf_classifier.feature_importances_

top_15_indices = np.argsort(feature_importances)[::-1][:15]

# 输出前15个最重要的特征及其重要性
print("Top 15 Most Important Features:")
l = []
for idx in top_15_indices:
    l.append(cancers.feature_names[idx])
    print(cancers.feature_names[idx], ":", feature_importances[idx])

l

Top 15 Most Important Features:
worst area : 0.15389236463205394
worst concave points : 0.14466326620735528
mean concave points : 0.10620998844591638
worst radius : 0.07798687515738047
mean concavity : 0.06800084191430111
worst perimeter : 0.06711483267839194
mean perimeter : 0.053269746128179675
mean radius : 0.048703371737755234
mean area : 0.04755500886018552
worst concavity : 0.031801595740040434
area error : 0.022406960160458473
worst texture : 0.021749011006763207
worst compactness : 0.020266035899623565
radius error : 0.02013891719419153
mean compactness : 0.013944325074050485


['worst area',
 'worst concave points',
 'mean concave points',
 'worst radius',
 'mean concavity',
 'worst perimeter',
 'mean perimeter',
 'mean radius',
 'mean area',
 'worst concavity',
 'area error',
 'worst texture',
 'worst compactness',
 'radius error',
 'mean compactness']

In [14]:
def random_forest_classification(X_train, X_test, y_train, y_test):
    from sklearn.datasets import load_breast_cancer
    from sklearn.model_selection import train_test_split
    from sklearn.feature_selection import RFE
    from sklearn.ensemble import RandomForestClassifier
    from sklearn.metrics import accuracy_score, recall_score
   # 1 调整max_depth
    score_all=[]
    num_RFC = []
    for i in range(1,10,1):
        clf_gini = RandomForestClassifier(criterion = 'gini', max_depth = i,random_state = 0)
        clf_gini.fit(X_train, y_train) # train
        y_pred_gini = clf_gini.predict(X_test) # predict
        y_pred_gini[0:5] # predict
        from sklearn.metrics import accuracy_score
        acc = recall_score(y_test, y_pred_gini)
        score_all.append([i, acc])
    ScoreAll = np.array(score_all)
    max_score = np.where(ScoreAll == np.max(ScoreAll[:,1]))[0][0]
    num_RFC.append(int(ScoreAll[max_score][0]))
    # print("best_score:", ScoreAll[max_score][1])
    # plt.figure(figsize=[20,5])
    # plt.plot(ScoreAll[:,0], ScoreAll[:,1])
    # plt.savefig(path_RFC + '/调参max_depth.png', dpi= 400)
    # plt.show()

    # 2 调整min_samples_split
    score_all=[]
    for i in range(2, 10, 1):
        clf_gini = RandomForestClassifier(criterion = 'gini', max_depth = num_RFC[0], min_samples_split = i,random_state=0)
        clf_gini.fit(X_train, y_train) # train
        y_pred_gini = clf_gini.predict(X_test) # predict
        y_pred_gini[0:5] # predict
        from sklearn.metrics import accuracy_score
        acc = recall_score(y_test, y_pred_gini)
        score_all.append([i, acc])
    ScoreAll = np.array(score_all)
    max_score = np.where(ScoreAll == np.max(ScoreAll[:,1]))[0][0]
    num_RFC.append(int(ScoreAll[max_score][0]))
    # print("best_score:", ScoreAll[max_score][1])
    # plt.figure(figsize=[20,5])
    # plt.plot(ScoreAll[:,0], ScoreAll[:,1])
    # plt.savefig(path_RFC + '/调参min_samples_split.png', dpi= 400)
    # plt.show()

    # 3 调整min_samples_leaf
    score_all=[]
    for i in range(1,10,1):
        clf_gini = RandomForestClassifier(criterion = 'gini', max_depth = num_RFC[0], min_samples_split = num_RFC[1], min_samples_leaf = i,random_state=0)
        clf_gini.fit(X_train, y_train) # train
        y_pred_gini = clf_gini.predict(X_test) # predict
        y_pred_gini[0:5] # predict
        from sklearn.metrics import accuracy_score
        acc = recall_score(y_test, y_pred_gini)
        score_all.append([i, acc])
    ScoreAll = np.array(score_all)
    max_score = np.where(ScoreAll == np.max(ScoreAll[:,1]))[0][0]
    num_RFC.append(int(ScoreAll[max_score][0]))
    # print("best_score:", ScoreAll[max_score][1]) 
    # plt.figure(figsize=[20,5])
    # plt.plot(ScoreAll[:,0], ScoreAll[:,1])
    # plt.savefig(path_RFC + '/调参min_samples_leaf.png', dpi= 400)
    # plt.show()
    print("best_num:", num_RFC)

    RFC = RandomForestClassifier(criterion = 'gini', max_depth = num_RFC[0], min_samples_split = num_RFC[1], min_samples_leaf = num_RFC[2],random_state=0)
    RFC.fit(X_train, y_train)

    y_pred_RFC = RFC.predict(X_test) # predict
    # 评估
    from sklearn.metrics import classification_report, mean_absolute_error, accuracy_score, mean_squared_error, f1_score
    score = accuracy_score(y_test, y_pred_RFC)
    recall = recall_score(y_test, y_pred_RFC)
    print("实际分类：", y_test)
    print("预测分类：", y_pred_RFC)
    print("准确率：{:.2%}".format(score))
    print("召回率：{:.2%}".format(recall))

In [16]:
from sklearn.metrics import accuracy_score, recall_score

# 基于前10个最重要的特征创建特征选择后的训练集和测试集
selected_cancers_X_train = cancers_X_train[:, top_10_indices]
selected_cancers_X_test = cancers_X_test[:, top_10_indices]

# 在选择的特征上进行训练和预测
rf_classifier.fit(selected_cancers_X_train, cancers_Y_train)
cancers_Y_pred = rf_classifier.predict(selected_cancers_X_test)
random_forest_classification(selected_cancers_X_train, selected_cancers_X_test, cancers_Y_train, cancers_Y_pred)

best_num: [1, 2, 1]
实际分类： [1 0 0 1 1 0 0 0 0 1 1 0 1 0 1 0 1 1 1 0 1 1 0 1 1 1 1 1 1 0 1 1 1 1 1 1 0
 1 0 1 1 0 1 1 1 1 1 1 1 1 0 0 1 1 1 1 1 0 0 1 1 0 0 1 1 1 0 0 1 1 0 0 1 0
 1 1 1 1 1 1 0 1 1 0 0 0 0 0 1 1 1 1 1 1 1 1 0 0 1 0 0 1 0 0 1 1 1 0 0 1 0
 1 1 0]
预测分类： [1 0 0 1 1 0 0 0 0 1 1 0 1 0 1 0 1 1 1 0 1 1 0 1 1 1 1 1 1 0 1 1 1 1 1 1 0
 1 0 1 1 0 1 1 1 1 1 1 1 1 0 0 1 1 1 1 1 0 1 1 1 0 0 1 1 1 0 0 1 1 0 0 1 0
 1 1 1 1 1 1 0 1 1 0 0 0 0 0 1 1 1 1 1 1 1 1 0 0 1 0 0 1 0 0 1 1 1 0 1 1 0
 1 1 0]
准确率：98.25%
召回率：100.00%


In [4]:
cancers.feature_names

array(['mean radius', 'mean texture', 'mean perimeter', 'mean area',
       'mean smoothness', 'mean compactness', 'mean concavity',
       'mean concave points', 'mean symmetry', 'mean fractal dimension',
       'radius error', 'texture error', 'perimeter error', 'area error',
       'smoothness error', 'compactness error', 'concavity error',
       'concave points error', 'symmetry error',
       'fractal dimension error', 'worst radius', 'worst texture',
       'worst perimeter', 'worst area', 'worst smoothness',
       'worst compactness', 'worst concavity', 'worst concave points',
       'worst symmetry', 'worst fractal dimension'], dtype='<U23')

In [5]:
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression

# 创建逻辑回归模型作为特征选择器
estimator = LogisticRegression(solver='liblinear')
# 创建RFE对象
rfe = RFE(estimator, n_features_to_select=10)  # 选择最重要的10个特征

# 在训练数据上进行特征选择
selected_features = rfe.fit_transform(cancers_X_train, cancers_Y_train)

# 获取选择的特征索引
feature_indices = rfe.get_support(indices=True)

# 根据选择的特征索引提取训练和测试数据集的子集
selected_cancers_X_train = cancers_X_train[:, feature_indices]
selected_cancers_X_test = cancers_X_test[:, feature_indices]

# 输出选择的特征数量
print("Selected Features:", len(feature_indices))

Selected Features: 10


> 使用递归特征消除法进行数据选择

In [20]:
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import RFE
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, recall_score

# 加载乳腺癌数据集
cancers = load_breast_cancer()
# 划分训练集和测试集
cancers_X_train, cancers_X_test, cancers_Y_train, cancers_Y_test = train_test_split(cancers.data, cancers.target, test_size=0.2, random_state=42)
# 创建逻辑回归模型作为特征选择器
estimator = RandomForestClassifier(n_estimators=100, random_state=42)

rfe = RFE(estimator, n_features_to_select=15)  # 选择最重要的15个特征
# 在训练数据上进行特征选择
selected_features = rfe.fit_transform(cancers_X_train, cancers_Y_train)
# 获取选择的特征索引
feature_indices = rfe.get_support(indices=True)

# 根据选择的特征索引提取训练和测试数据集的子集
selected_cancers_X_train = cancers_X_train[:, feature_indices]
selected_cancers_X_test = cancers_X_test[:, feature_indices]

# 输出选择的特征数量和特征名称
print("Selected Features:", len(feature_indices))
print("Selected Feature Names:")
l = []
for idx in feature_indices:
    l.append(cancers.feature_names[idx])
    print(cancers.feature_names[idx])
print(l)

# 创建随机森林分类器对象
rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)

# 在选择的特征上进行训练和预测
rf_classifier.fit(selected_cancers_X_train, cancers_Y_train)
cancers_Y_pred = rf_classifier.predict(selected_cancers_X_test)

def random_forest_classification(X_train, X_test, y_train, y_test):
   # 1 调整max_depth
    score_all=[]
    num_RFC = []
    for i in range(1,10,1):
        clf_gini = RandomForestClassifier(criterion = 'gini', max_depth = i,random_state = 0)
        clf_gini.fit(X_train, y_train) # train
        y_pred_gini = clf_gini.predict(X_test) # predict
        y_pred_gini[0:5] # predict
        from sklearn.metrics import accuracy_score
        acc = recall_score(y_test, y_pred_gini)
        score_all.append([i, acc])
    ScoreAll = np.array(score_all)
    max_score = np.where(ScoreAll == np.max(ScoreAll[:,1]))[0][0]
    num_RFC.append(int(ScoreAll[max_score][0]))
    # print("best_score:", ScoreAll[max_score][1])
    # plt.figure(figsize=[20,5])
    # plt.plot(ScoreAll[:,0], ScoreAll[:,1])
    # plt.savefig(path_RFC + '/调参max_depth.png', dpi= 400)
    # plt.show()

    # 2 调整min_samples_split
    score_all=[]
    for i in range(2, 10, 1):
        clf_gini = RandomForestClassifier(criterion = 'gini', max_depth = num_RFC[0], min_samples_split = i,random_state=0)
        clf_gini.fit(X_train, y_train) # train
        y_pred_gini = clf_gini.predict(X_test) # predict
        y_pred_gini[0:5] # predict
        from sklearn.metrics import accuracy_score
        acc = recall_score(y_test, y_pred_gini)
        score_all.append([i, acc])
    ScoreAll = np.array(score_all)
    max_score = np.where(ScoreAll == np.max(ScoreAll[:,1]))[0][0]
    num_RFC.append(int(ScoreAll[max_score][0]))
    # print("best_score:", ScoreAll[max_score][1])
    # plt.figure(figsize=[20,5])
    # plt.plot(ScoreAll[:,0], ScoreAll[:,1])
    # plt.savefig(path_RFC + '/调参min_samples_split.png', dpi= 400)
    # plt.show()

    # 3 调整min_samples_leaf
    score_all=[]
    for i in range(1,10,1):
        clf_gini = RandomForestClassifier(criterion = 'gini', max_depth = num_RFC[0], min_samples_split = num_RFC[1], min_samples_leaf = i,random_state=0)
        clf_gini.fit(X_train, y_train) # train
        y_pred_gini = clf_gini.predict(X_test) # predict
        y_pred_gini[0:5] # predict
        from sklearn.metrics import accuracy_score
        acc = recall_score(y_test, y_pred_gini)
        score_all.append([i, acc])
    ScoreAll = np.array(score_all)
    max_score = np.where(ScoreAll == np.max(ScoreAll[:,1]))[0][0]
    num_RFC.append(int(ScoreAll[max_score][0]))
    # print("best_score:", ScoreAll[max_score][1]) 
    # plt.figure(figsize=[20,5])
    # plt.plot(ScoreAll[:,0], ScoreAll[:,1])
    # plt.savefig(path_RFC + '/调参min_samples_leaf.png', dpi= 400)
    # plt.show()
    print("best_num:", num_RFC)

    RFC = RandomForestClassifier(criterion = 'gini', max_depth = num_RFC[0], min_samples_split = num_RFC[1], min_samples_leaf = num_RFC[2],random_state=0)
    RFC.fit(X_train, y_train)

    y_pred_RFC = RFC.predict(X_test) # predict
    # 评估
    from sklearn.metrics import classification_report, mean_absolute_error, accuracy_score, mean_squared_error, f1_score
    score = accuracy_score(y_test, y_pred_RFC)
    recall = recall_score(y_test, y_pred_RFC)
    print("实际分类：", y_test)
    print("预测分类：", y_pred_RFC)
    print("准确率：{:.2%}".format(score))
    print("召回率：{:.2%}".format(recall))

random_forest_classification(selected_cancers_X_train, selected_cancers_X_test, cancers_Y_train, cancers_Y_pred)

Selected Features: 15
Selected Feature Names:
mean radius
mean texture
mean perimeter
mean area
mean concavity
mean concave points
worst radius
worst texture
worst perimeter
worst area
worst smoothness
worst compactness
worst concavity
worst concave points
worst symmetry
['mean radius', 'mean texture', 'mean perimeter', 'mean area', 'mean concavity', 'mean concave points', 'worst radius', 'worst texture', 'worst perimeter', 'worst area', 'worst smoothness', 'worst compactness', 'worst concavity', 'worst concave points', 'worst symmetry']
best_num: [1, 2, 1]
实际分类： [1 0 0 1 1 0 0 0 0 1 1 0 1 0 1 0 1 1 1 0 1 1 0 1 1 1 1 1 1 0 1 1 1 1 1 1 0
 1 0 1 1 0 1 1 1 1 1 1 1 1 0 0 1 1 1 1 1 0 0 1 1 0 0 1 1 1 0 0 1 1 0 0 1 0
 1 1 1 1 1 1 0 1 1 0 0 0 0 0 1 1 1 1 1 1 1 1 0 0 1 0 0 1 0 0 1 1 1 0 1 1 0
 1 1 0]
预测分类： [1 0 0 1 1 0 0 0 1 1 1 0 1 0 1 0 1 1 1 0 1 1 0 1 1 1 1 1 1 0 1 1 1 1 1 1 0
 1 0 1 1 0 1 1 1 1 1 1 1 1 0 0 1 1 1 1 1 0 1 1 1 0 0 1 1 1 0 0 1 1 1 0 1 0
 1 1 1 1 1 1 0 1 1 0 0 0 1 0 1 1 1 1 1 1 